In [ ]:
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/fashion/", one_hot=True)
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Run this if you want to train the network on your own machine
# (This takes a while)

tf.reset_default_graph()

def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x, name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

x = tf.placeholder(tf.float32, shape=[None, 784], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 10], name='y_')

W = tf.Variable(tf.zeros([784,10]), name='W')
b = tf.Variable(tf.zeros([10]), name='b')

W_conv1 = weight_variable([5, 5, 1, 32], 'W_conv1')
b_conv1 = bias_variable([32], 'b_conv1')
x_image = tf.reshape(x, [-1,28,28,1], 'x_image')
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1, name='h_conv1')
h_pool1 = max_pool_2x2(h_conv1, 'h_pool1')

W_conv2 = weight_variable([5, 5, 32, 64], 'W_conv2')
b_conv2 = bias_variable([64], 'b_conv2')
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2, name='h_conv2')
h_pool2 = max_pool_2x2(h_conv2, 'h_pool2')

W_fc1 = weight_variable([7 * 7 * 64, 1024], 'W_fc1')
b_fc1 = bias_variable([1024], 'b_fc1')
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name='h_pool2_flat')
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name='h_fc1')

keep_prob = tf.placeholder(tf.float32, name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name='h_fc1_drop')

W_fc2 = weight_variable([1024, 10], 'W_fc2')
b_fc2 = bias_variable([10], 'b_fc2')
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name="output")

sess = tf.InteractiveSession()

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]), name='cross_entropy')
train_step = tf.train.AdamOptimizer(1e-4, name='train_step').minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1), name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

sess.run(tf.global_variables_initializer())

# Putting this in a function to use later when we want to re-train

for i in range(10000):
    batch = mnist.train.next_batch(1000)
    if i%1000 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g" % sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

saver = tf.train.Saver()
save_path = saver.save(sess, "models/mnist_cnn.ckpt")
print("Model saved to %s" % save_path)

In [ ]:
print("test accuracy %g" % sess.run(accuracy, feed_dict={
    x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0}))

In [ ]:
# Define the classification we're optimizing for
digit = 0

# This is the probability that an input image 
returned_classification = y_conv[:, digit]

# Here's our image of random noise going in
img_noise = np.random.uniform(size=(1, 784))

# Let's take a look at it and see how the network classifies it now.
plt.imshow(img_noise.reshape([28, 28]), cmap='Greys')
#print(y_conv.eval({x: img_noise, keep_prob: 1}))


print(np.argmax(y_conv.eval({x: img_noise, keep_prob: 1})))

In [ ]:
gradient_op = tf.gradients(returned_classification, x)

for i in range(50):
    gradients = sess.run(gradient_op, {x: img_noise, keep_prob: 1})

    '''
    gradients now contains a 28 x 28 matrix. Each element is the derivative of
    the chosen classification with respect to a pixel in the image.
    '''

    gradients = gradients / (np.std(gradients) + 1e-8) # Normalize the gradients...
    img_noise = img_noise + gradients[0] # ...and add the normalized gradients to the image

plt.imshow(img_noise.reshape([28, 28]), cmap='Greys')
plt.show()

# Now, classify the adjusted img_noise
print(np.argmax(y_conv.eval({x: img_noise, keep_prob: 1})))

